# Eletrônica de Potência II - Tarefa 1

Alunos:
* Alejo Perdomo Milar
* João Mário Carnieletto Izoton Lago

# Conversor Buck com seguintes especificações:

* Tensão de Entrada: 36 V
* Tensão de Saída: 12 V
* Freq. de comutação: 100 kHz;
* Indutância L: 100 uH
* Capacitância C: 100 uF;
* Potência: 100 W;

# Circuito Simulado no LTSpice (Modelo Idealizado)

A imagem a seguir apresenta o circuito simulado no software LTSpice, utilizando componentes ideais para fins de análise.

![Schematic](./prints/schematic.png)

---

## Formas de Onda Típicas

A seguir, são apresentadas as formas de onda obtidas a partir da simulação, destacando a corrente no indutor, a potência dissipada na carga, bem como a tensão e a corrente na carga.

![typical](./prints/typical.png)

---

## Estresses na Chave

A imagem abaixo ilustra as formas de ondas relacionadas à chave. São exibidas a tensão de controle (Vgs), a tensão aplicada sobre a chave e a corrente nela.

![switch](./prints/switch.png)

A imagem seguinte apresenta um zoom da anterior, evidenciando os pontos de corrente máxima e mínima na chave.

![switch_zoom](./prints/switch_zoom.png)

Por fim, a figura abaixo mostra os valores médio e eficaz (RMS) da corrente que na chave.

![I_SW1](./prints/I_SW1.png)

---

## Estresses no Diodo

A figura a seguir apresenta as formas de onda relacionadas ao diodo. São mostradas a tensão de controle da chave, a tensão sobre o diodo e a corrente nele.

![diode](./prints/diode.png)

A imagem seguinte amplia a anterior, permitindo a observação dos instantes de corrente máxima e mínima no diodo.

![diode_zoom](./prints/diode_zoom.png)

A última imagem desta seção apresenta os valores médio e eficaz (RMS) da corrente no diodo.

![I_D1](./prints/I_D1.png)


---

# Estresses nos Componentes

## Perdas em condução e comutação na chave (IRLZ44N)

In [2]:
# Converter
duty = 0.34
freq = 100e3

# Switch
i_s_rms = 4.8242
i_s_on = 8.7445
v_ds_off = 36.0
switch_r_ds_on = 1.5 * 0.022 # @ 100C

# Assuming gate driver can instantly turn on and off the switch
switch_tr = 84e-9
switch_tf = 15e-9

# Switch losses
switch_conduction_loss = switch_r_ds_on * (i_s_rms**2)
switch_commutation_loss = (freq / 2) * (switch_tr + switch_tf) * (v_ds_off * i_s_on)

switch_total_loss = switch_conduction_loss + switch_commutation_loss

print(f"Perdas de condução na chave: {switch_conduction_loss:.2f} W")
print(f"Perdas de comutação na chave: {switch_commutation_loss:.2f} W")
print(f"Perdas totais na chave: {switch_total_loss:.2f} W")


Perdas de condução na chave: 0.77 W
Perdas de comutação na chave: 1.56 W
Perdas totais na chave: 2.33 W


---

## Perdas para o diodo MUR860 (Silicon Ultra Fast)

In [5]:
# diode
i_d_avg = 5.55
i_d_rms = 6.8122

mur_diode_v_threshold = 0.47
mur_diode_rt = 1.0 / 11.78 # Around 84.88mΩ for MUR860
mur_diode_q_rr = 195e-9
mur_diode_v_off = 36

mur_diode_conduction_loss = mur_diode_v_threshold * i_d_avg + mur_diode_rt * (i_d_rms*2)
mur_diode_commutation_loss = mur_diode_q_rr * mur_diode_v_off * freq

mur_diode_total_loss = mur_diode_conduction_loss + mur_diode_commutation_loss

print(f"Perdas de condução no diodo: {mur_diode_conduction_loss:.2f} W")
print(f"Perdas de comutação no diodo: {mur_diode_commutation_loss:.2f} W")
print(f"Perdas totais no diodo: {mur_diode_total_loss:.2f} W")

Perdas de condução no diodo: 3.77 W
Perdas de comutação no diodo: 0.70 W
Perdas totais no diodo: 4.47 W


---

## Perdas para o diodo MBR20100CT (Schottky)

In [6]:
# diode
i_d_avg = 5.55
i_d_rms = 6.8122

# Assuming Tj = 25 since data sheet only provides values for 25 and 150 and 25 is the worse case
mbr_diode_v_threshold = 0.3
mbr_diode_rt = 1.0 / 17.75555 # Around 58.8mΩ for MBR20100CT

mbr_diode_conduction_loss = mbr_diode_v_threshold * i_d_avg + mbr_diode_rt * (i_d_rms*2)
# Disconsider it since it is a Schottky diode
mbr_diode_commutation_loss = 0.0

mbr_diode_total_loss = mbr_diode_conduction_loss + mbr_diode_commutation_loss

print(f"Perdas de condução no diodo: {mbr_diode_conduction_loss:.2f} W")
print(f"Perdas de comutação no diodo: {mbr_diode_commutation_loss:.2f} W")
print(f"Perdas totais no diodo: {mbr_diode_total_loss:.2f} W")

Perdas de condução no diodo: 2.43 W
Perdas de comutação no diodo: 0.00 W
Perdas totais no diodo: 2.43 W


# Cálculo de dissipadores

In [7]:

def calculate_rth_max(diode_name, switch_temp_max, switch_r_junction_sink, switch_total_loss,
					  diode_temp_max, diode_r_junction_case, diode_total_loss,
					  ambient_temp):
	print(f"============ Calculo das perdas totais para a chave IRLZ44N e o diodo {diode_name} ============")

	switch_temp_sink_max = switch_temp_max - (switch_r_junction_sink * switch_total_loss)
	print(f"Temperatura maxima no dissipador para a chave: {switch_temp_sink_max:.2f} C")

	diode_temp_sink_max = diode_temp_max - (diode_r_junction_case * diode_total_loss)
	print(f"Temperatura maxima no dissipador para o diodo: {diode_temp_sink_max:.2f} C")

	total_power_loss = switch_total_loss + diode_total_loss
	print(f"Perdas totais: {total_power_loss:.2f} W")

	heatsink_r_max = (min([diode_temp_sink_max, switch_temp_sink_max]) - ambient_temp) / total_power_loss

	print(f"Rth maximo do dissipador: {heatsink_r_max:.2f} C/W")


# IRLZ44N
switch_r_junction_case = 1.4
switch_r_case_sink = 0.5
switch_r_junction_sink = switch_r_junction_case + switch_r_case_sink
switch_r_junction_ambient = 62.0
switch_temp_max = 175.0 * 0.8 # Assuming 20% tolerance

# MUR805
mur_diode_r_junction_case = 3.0
mur_diode_temp_max = 175.0 * 0.8 # Assuming 20% tolerance

# MBR20100
mbr_diode_r_junction_case = 2.0
mbr_diode_temp_max = 150.0 * 0.8 # Assuming 20% tolerance

# Assuming 70C ambient temperature
ambient_temp = 70.0

calculate_rth_max("MUR805", switch_temp_max, switch_r_junction_sink, switch_total_loss,
				  mur_diode_temp_max, mur_diode_r_junction_case, mur_diode_total_loss,
				  ambient_temp)

calculate_rth_max("MBR20100", switch_temp_max, switch_r_junction_sink, switch_total_loss,
				  mbr_diode_temp_max, mbr_diode_r_junction_case, mbr_diode_total_loss,
				  ambient_temp)


============ Calculo das perdas totais para a chave IRLZ44N e o diodo MUR805 ============
Temperatura maxima no dissipador para a chave: 135.58 C
Temperatura maxima no dissipador para o diodo: 126.60 C
Perdas totais: 6.79 W
Rth maximo do dissipador: 8.33 C/W
============ Calculo das perdas totais para a chave IRLZ44N e o diodo MBR20100 ============
Temperatura maxima no dissipador para a chave: 135.58 C
Temperatura maxima no dissipador para o diodo: 115.14 C
Perdas totais: 4.76 W
Rth maximo do dissipador: 9.48 C/W


## Dimensões físicas requeridas minimas

### Diomensões por componente

#### IRLZ44N

* Largura: 10.54 mm -> 11.5 mm com margem
* Altura: 15.24 mm -> 16.5 mm com margem

#### MUR860

* Largura: 10.29 mm -> 11.5 mm com margem
* Altura: 15.75 mm -> 16.5 mm com margem

#### MBR20100CT

* Largura: 10.5 mm -> 11.5 mm com margem
* Altura: 16 mm -> 16.5 mm com margem

### Dimensões físicas superficie de contato dissipador

#### Lado a Lado

* Largura: 11.5 mm + 11.5 mm = 23 mm
* Altura: 16.5 mm

#### Frente e verso

* Largura: 11.5 mm
* Altura: 16.5 mm

### Fabricante HS

#### Fator de Correção pelo comprimento
![hs_6520](./heatsink/hs_length_correction.png)

#### Detalhes do dissipador HS 5620
![hs_6520](./heatsink/hs_5620.png)

#### Utilizando comprimento de 20 mm

Resistência térmica: $2.9 \cdot 2.21 = 6.409 °C / W$

Aplicando uma tolerância de 20% dado a correção de $\Delta T$:

Resistência térmica: $6.409 \cdot 1.2 = 7.69 °C / W$

Dissipador é operacional para ambos os casos (IRLZ44N e MUR860) ou (IRLZ44N e MBR20100CT), dado que o valor do dissipador é relativamente barato, aproximadamente R$ 5,00, cotado em 2025.

### Fabricante ATS (Advanced Thermal Solutions)

Acesso [digikey](https://www.digikey.com/en/products/detail/advanced-thermal-solutions-inc/ATS-PCB1047/5030467)

#### Detalhes ATS PCB-1047

![ats_1047](./heatsink/ats_pcb_1047.png)

Resistência térmica: $8.0 °C / W$, comprimento único. Preço de aproximadamente R$ 13,00, cotado em 2025.
